In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

In [2]:
import matplotlib.pyplot as plt
from simulation3D import Simulation3D
from optimization3d import OptimizationManager3D
from config import (
    WAVE_FREQ, WAVE_SPEED, WAVE_DECAY, WAVE_CUTOFF, WAVE_RETREAT_COEFF, WAVE_HEIGHT,
    SAND_PULL, GROUND_PULL, WATER_DECAY, WAVE_VOL, WAVE_AMPLITUDE, WAVE_SPREAD,
    DIM_MAP, OBSTACLE_COORDS
)
import numpy as np

# Define simulation parameters
SIM_PARAMS = {
    "wave_freq": WAVE_FREQ,
    "wave_speed": WAVE_SPEED,
    "wave_decay": WAVE_DECAY,
    "wave_cutoff": WAVE_CUTOFF,
    "wave_retreat_coeff": WAVE_RETREAT_COEFF,
    "wave_height": WAVE_HEIGHT,
    "sand_pull": SAND_PULL,
    "ground_pull": GROUND_PULL,
    "water_decay": WATER_DECAY,
    "wave_vol": WAVE_VOL,
    "wave_amplitude": WAVE_AMPLITUDE,
    "wave_spread": WAVE_SPREAD,
    "dim_map": DIM_MAP
}

# 1. Run simulation without barriers
print("\nRunning 3D simulation without barriers...")
sim_no_barrier = Simulation3D(**SIM_PARAMS, obstacle_coords=[])
sim_no_barrier.run_sim(num_timesteps=10)

# 2. Run simulation with non-optimized barriers
print("\nRunning 3D simulation with non-optimized barriers...")
sim_non_optimized = Simulation3D(**SIM_PARAMS, obstacle_coords=OBSTACLE_COORDS)
sim_non_optimized.run_sim(num_timesteps=10)

# 3. Optimize barrier placement
print("\nOptimizing 3D barrier placement...")
def run_optimization_3d(num_timesteps=10, n_samples=10, n_iterations=20):
    opt_manager = OptimizationManager3D()
    param_samples = opt_manager.sample_parameters(n_samples)
    
    # Run initial simulations
    outcomes = []
    for params in param_samples:
        outcomes.append(opt_manager.wrapped_run_sim(params))
    
    # Convert results to numpy arrays
    X = np.array(param_samples)
    Y = np.array(outcomes).reshape(-1, 1)
    
    # Run Bayesian optimization
    loop_state = opt_manager.run_optimization(X, Y, n_iterations=n_iterations)
    
    # Extract the best parameters
    best_parameters = loop_state.X[-1]
    return [(int(best_parameters[i]), int(best_parameters[len(OBSTACLE_COORDS) + i])) for i in range(len(OBSTACLE_COORDS))]

optimized_coords = run_optimization_3d()

# 4. Run simulation with optimized barriers
print("\nRunning 3D simulation with optimized barriers...")
sim_optimized = Simulation3D(**SIM_PARAMS, obstacle_coords=optimized_coords)
sim_optimized.run_sim(num_timesteps=10)

# 5. Visualization (3D Voxels)
print("\nVisualizing 3D simulations...")
sim_no_barrier.plot_3d_downsamp()
sim_non_optimized.plot_3d_downsamp()
sim_optimized.plot_3d_downsamp()

# 6. Numerical summary
sand_no_barrier = np.sum(sim_no_barrier.coast_map[:, :, 1]) - np.sum(sim_no_barrier.coast_map_3D[:, :, :, 1])
sand_non_optimized = np.sum(sim_non_optimized.coast_map[:, :, 1]) - np.sum(sim_non_optimized.coast_map_3D[:, :, :, 1])
sand_optimized = np.sum(sim_optimized.coast_map[:, :, 1]) - np.sum(sim_optimized.coast_map_3D[:, :, :, 1])

print("\nErosion Summary (3D):")
print(f"Total erosion (no barriers): {sand_no_barrier:.2f}")
print(f"Total erosion (non-optimized barriers): {sand_non_optimized:.2f}")
print(f"Total erosion (optimized barriers): {sand_optimized:.2f}")



Running 3D simulation without barriers...


  0%|          | 0/99 [00:00<?, ?it/s]

KeyboardInterrupt: 